In [ ]:
# GPU는 무조건 T4라도 연결하고 사용하셔야 합니다!!!!!!!

In [32]:
import tensorflow as tf

In [6]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_y),(test_X, test_y) = fashion_mnist.load_data()
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [7]:
# 전처리 : 이미지 전처리 --> 1/255.0으로 정규화
train_X = train_X/255.0
test_X = test_X /255.0

In [8]:
train_X.shape

(60000, 28, 28)

In [9]:
train_X[0].shape # 2d

(28, 28)

In [10]:
print(train_X.shape)
print(train_X[0].shape)
train_X = train_X.reshape(-1 ,28,28,1)  # 3d ---> 4d
test_X  = test_X.reshape(-1, 28,28,1)
print(train_X.shape)
print(train_X[0].shape)
# 목적 : 모델의 구조도에 맞춰서 데이터셋을 변형!!!!!

(60000, 28, 28)
(28, 28)
(60000, 28, 28, 1)
(28, 28, 1)


In [11]:
### VGG 선행 연구 결과를 참조

<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16.png">

In [12]:
https://arxiv.org/pdf/1409.1556

SyntaxError: invalid syntax (<ipython-input-12-d63fc2938d6b>, line 1)

In [14]:
# - 기존의 설계 방식에서는 Conv - Pool 구조를 이어서 사용
#   ==> Conv-Conv-Pool/ Conv-Conv-Conv-Pool
# 기존에는 대략 큰 필터들을 사용을 했음 7*7, 9*9, 11*11 을 사용했음

# 10*10에서 7*7(필터) 하면 49C2 연산이 필요, 결과는 4*4
# 10*10에서 3*3(필터) 하면 9C2 연산이 필요, 결과는 4*4
# 결과물이 똑같지만 연산량이 확 줄기 때문에 적층을 할 수 있음
# + AF 집어넣은 것들은 1번 보다 3번 통과해서 더 비선형을 획득할 수 있음

In [15]:
# 논문에서 연산도 줄고, 잘 된다고 하고, 내가 원하면 더 층을 쌓을 수도 있는
# 응용부분도 존재 : 튜닝이 쉬움
# 기본 구조 : 늘 했던 단순 적층 구조


In [21]:
# 코드가 길어지니 간략하게 불러오겠음
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [17]:
# 기본적으로 VGG 설계 방식 차용
# -> 입력 데이터가 다르기 때문
# 논문 상 입력 : 224, 224, 3
# 나의 데이터 :28, 28, 1
# sol1) 논문상 데이터 크기 맞추기
# sol2) 이해했을 경우, 상화엥 맞춰 설계

In [26]:
model_vgg_fashion = tf.keras.Sequential(
    [
        # 원본 입력을 기준으로 설계.... : 224 224 RGB
        # --> 28 28 1
        Conv2D( input_shape=(28,28,1), kernel_size=(3,3),
               filters=64, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=64, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters=128, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=128, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),
        # ===> 기본적인 이미지가 가지고 있는 특징 추출!!!!!

        # 분류를 위한 NN
        # 1) 특징을 분류 Faltten
        Flatten(),
        # 2) 분류용 HL 적층!!!!
        Dense( units=4096, activation="relu"),
        Dense( units=4096, activation="relu"),
        # 3) 출력용 --> 내 모델의 목적에 맞춰야함!!
        #    나는 10개만 하면 됨...수정..
        Dense( units=10, activation="softmax")
    ]
)
model_vgg_fashion

In [27]:
import os

In [28]:
# --> epochs를 일단 크게 세팅을 하고,,,
#     early stop을 통해서 더이상 성능 향상이 없으면 stop
# --> 중간 중간 모델의 weights를 저장(모델 저장)
cp_path = "training/cp-{epoch:04d}.ckpt"
cp_dir = os.path.dirname(cp_path) # training
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    cp_path,
    verbose = 1,
    save_weights_only = True
)
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience = 10 # 참을 횟수...갱신이 안 되는 횟수..
)

model_vgg_fashion.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = "sparse_categorical_crossentropy",
    metrics =["accuracy"]
)

history = model_vgg_fashion.fit( train_X, train_y,
                    epochs = 200,
                     validation_split = 0.25,
                     batch_size=128,
                     callbacks=[cp_callback,es_callback])

Epoch 1/200
352/352 [==============================] - ETA: 0s - loss: 0.7661 - accuracy: 0.6984
Epoch 1: saving model to training/cp-0001.ckpt
352/352 [==============================] - 22s 50ms/step - loss: 0.7661 - accuracy: 0.6984 - val_loss: 0.4262 - val_accuracy: 0.8440
Epoch 2/200
351/352 [============================>.] - ETA: 0s - loss: 0.3428 - accuracy: 0.8753
Epoch 2: saving model to training/cp-0002.ckpt
352/352 [==============================] - 16s 44ms/step - loss: 0.3429 - accuracy: 0.8753 - val_loss: 0.3160 - val_accuracy: 0.8857
Epoch 3/200
351/352 [============================>.] - ETA: 0s - loss: 0.2832 - accuracy: 0.8978
Epoch 3: saving model to training/cp-0003.ckpt
352/352 [==============================] - 15s 44ms/step - loss: 0.2832 - accuracy: 0.8977 - val_loss: 0.2889 - val_accuracy: 0.8978
Epoch 4/200
351/352 [============================>.] - ETA: 0s - loss: 0.2515 - accuracy: 0.9081
Epoch 4: saving model to training/cp-0004.ckpt
352/352 [================

In [29]:
model_vgg_fashion.evaluate(test_X, test_y)

313/313 [==============================] - 2s 6ms/step - loss: 0.3862 - accuracy: 0.9230


[0.386211633682251, 0.9229999780654907]

In [ ]:
# ++conv 필터 사이즈, conv 필터 개수, conv layer 깊이
# 분류에 있는 dense 깊이, 노드, ac
# ++ dropout
# 실험해보기

In [ ]:
# cnn 에서 이미지 분류 쪽에서 주로 개선방향
# 1) 최신 모델 구조 차용
# 2) 데이터 수가 모델의 파라미터 수 대비 부족
# => 가상의 데이터를 생성해서 이미지를 보충, 입력 데이터의 샘플 수를 엄청 늘리는 직업
# agumentation 이미지 증강, 보강 -> 성능 상승!

In [ ]:
# try1) 모델의 구조
# try2) 데이터의 수! + 직접 수집을 더 하면 됨
# -> 수집한 데이터를 기반으로 가상의 이미지를 생성
# (bagging -> bootstraping 유사한 데이터셋을 여러개 생성!)
# 원본과 거의 유사하지만 다른 가상의 데이터 생성 (이미지 보강, 증강)

# 활용방법1) 직접 이미지를 쌓아서 처리 : (60000,28,28,1)->(90000,28,28,1)
#           입력 데이터가 그대로 존재해서 코드상 손댈 부분. 굳이 HDD 차지할 필요가 있을까?
# 활용방법2) 학습할 때만 생성을 하고, 학습 후에는 제거!
#           -> 원천 소스가 되는 이미지느 그대로 두고, 파생시킨 이미지들은

In [38]:
rotation_range = 10

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 10, # 단, 너무 상하 반전이 이미지도 생성이 될 수 있음
    zoom_range=0.10, # 너무 커서 넘어가거나, 너무 작아서 안보이거나
    horizontal_flip = True,
    vertical_flip = False,
    # etc

)

# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
# 참고) 직접 만들어서 추가하는 것은 생략 (거의 사용 안함)
# img_gen.flow~~

In [ ]:
# 위의 generator를 기반으로 hdd는 하지 않지만,
# 만들 수 있는 generator에 대한 세팅!!!!!!
# ==> batch단위로 처리를 함!!!!
# ( 폴더 중심으로 세팅을 하는게 일반적인데,,,,주어진 셋으로 )
# ==> yolo 모델을 가지고 할 때는 폴더 중심으로 하겠습니다..

In [41]:
# 데이터에 대한 셋도 미리 만들어 둬야 함
# 핵심은 직접 train/val 생성을 통해서 만들어 주어야 함

train_batches = img_gen.flow(train_X, train_y, batch_size = 256)
val_batches = img_gen.flow(test_X, test_y, batch_size = 256)

# --> val에 평가하는 이미지가 난이도가 높아짐
# --> 각각 나눠서 생성을 하는 경우가 있음.(지금은 그냥 코드가 길어지니깐 한번에 처리)

In [42]:
model_vgg_fashion = tf.keras.Sequential(
    [
        # 원본 입력을 기준으로 설계.... : 224 224 RGB
        # --> 28 28 1
        Conv2D( input_shape=(28,28,1), kernel_size=(3,3),
               filters=64, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=64, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters=128, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=128, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters=256, padding ="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters=512, padding ="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),
        # ===> 기본적인 이미지가 가지고 있는 특징 추출!!!!!

        # 분류를 위한 NN
        # 1) 특징을 분류 Faltten
        Flatten(),
        # 2) 분류용 HL 적층!!!!
        Dense( units=4096, activation="relu"),
        Dense( units=4096, activation="relu"),
        # 3) 출력용 --> 내 모델의 목적에 맞춰야함!!
        #    나는 10개만 하면 됨...수정..
        Dense( units=10, activation="softmax")
    ]
)
model_vgg_fashion

In [43]:
# --> epochs를 일단 크게 세팅을 하고,,,
#     early stop을 통해서 더이상 성능 향상이 없으면 stop
# --> 중간 중간 모델의 weights를 저장(모델 저장)
cp_path = "training/cp-{epoch:04d}.ckpt"
cp_dir = os.path.dirname(cp_path) # training
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    cp_path,
    verbose = 1,
    save_weights_only = True
)
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience = 10 # 참을 횟수...갱신이 안 되는 횟수..
)

model_vgg_fashion.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = "sparse_categorical_crossentropy",
    metrics =["accuracy"]
)

# 검색했을 때 fit_generator : 이미지 증강
# --> fit을 통합할 거임

# pytorch 를 무조건 데이셋을 중심으로 핸들링을 함!!!
# ==> 생성기를 통해서 처리하는 코드 스타일이 일반적!!!
# pytorch로 실습할 때 설명을 드리겠습니다..class + decortor + template etc
# 프로그램으로 -p.py중심으로 구성을 해야함!!!
# 모듈화!!!!

# fit으로 통합해서 혼동 발생 가능
# X, y 각각 넣어서 학습했는지, 덩어리로 처리했는지에 따라
# 파라미터 옵션이 달라지니 확인해야함.

history = model_vgg_fashion.fit(train_batches, # 성기를 중심으로 셋.셋 생성기
                    epochs = 200,
                    validation_data = val_batches,
                    callbacks=[cp_callback,es_callback])

Epoch 1/200
235/235 [==============================] - ETA: 0s - loss: 0.7590 - accuracy: 0.7115
Epoch 1: saving model to training/cp-0001.ckpt
235/235 [==============================] - 37s 131ms/step - loss: 0.7590 - accuracy: 0.7115 - val_loss: 0.4546 - val_accuracy: 0.8300
Epoch 2/200
235/235 [==============================] - ETA: 0s - loss: 0.3914 - accuracy: 0.8509
Epoch 2: saving model to training/cp-0002.ckpt
235/235 [==============================] - 23s 99ms/step - loss: 0.3914 - accuracy: 0.8509 - val_loss: 0.3587 - val_accuracy: 0.8633
Epoch 3/200
235/235 [==============================] - ETA: 0s - loss: 0.3150 - accuracy: 0.8824
Epoch 3: saving model to training/cp-0003.ckpt
235/235 [==============================] - 24s 100ms/step - loss: 0.3150 - accuracy: 0.8824 - val_loss: 0.3363 - val_accuracy: 0.8835
Epoch 4/200
235/235 [==============================] - ETA: 0s - loss: 0.2779 - accuracy: 0.8959
Epoch 4: saving model to training/cp-0004.ckpt
235/235 [==============

In [ ]:
# 딥러닝 많은 데이터가 필요함. 성능에 비례
# CNN 구조는 더 ㅁ낳은 파라미터들을 요구함
# -> 더 많은 데이터가 필요한 부분이 따라옴
# 1) 직접 수집의 양을 늘리거나
# 2) 직접 내가 가상의 이미지를 생성해서 늘리거나,
# 3)이미 잘 학습된 특징 추출에 대한 모델이 있어서, 그거 가져다가 사용을 함
# (모델이 깊고, 학습 데이터도 방대함)
# -> 나의 데이터 셋으로 Fine tuning 을 하는 방식(데이터 증강)
# 이미지 데이터 셋 : hdd 보관을 할 수 없음!
# -> 메모리에 모든 셋을 저장할 수 없음!
# -> 메모리에 다 올라가면 빠르게 처리는 가능하나 불가능함

# 이미지 저장은 나의 local/ HDD
# 필요할 때마다 가지고 와서 모델에 토스(generator)
# +batch_size(필수)
# 좀 데이터를 증강할까 (옵션)
# -> HDD - RAM 지연 발생, 학습 속도가 조금 느림
# 본인 HW 상태를 모니터링 하면서 batch_size를 조절하실 필요 있음
# 나중에 클라우드. 개인 워크스테이션에서 할 때 모델을 다 저장하면
# 1개 모델의 용량이 크니, 나의 hdd 가 부족할 수 있음. 잘 관리 해야함
# (클라우드: 클라우드가 멈춰 복구 불가능...
#  미리 세팅ㅇ르 hdd 여유롭게 잡고 진행을 해야함)

# DNN ---> CNN ----> VGG + 데이터 증강
# 앞으로의 모델은 데이터 증강 사용안 하고,,모델 중심으로

# 실제 kaggle 셋을 할 때에는 이미지 file + 폴더 구조 + 생성기(+ 증강)